# Test kmax impact

kmax has a large impact on the memory so we want to test if we can set it to 1 without problem.

### Memory stored
The shape of the TOD is (Nsamples, Ntes).

And you have say 9 peaks in the synthesized beam model (corresponding to kmax=1) so the size of the operators is indeed: (Ntod, Npeaks)

Note it is **independent of Nside**. This is because you don't need to store the H operator which is in the map space. You just want to store the location of each peak at each time sample.

So what is stored is the pixel number of each peak for each time sample and each TES.

**Example:** 999 ptgs, 992 TES (FI), kmax=2 meaning 25 peaks

        992 * 999 = 991008
        
        then if you take 16 bits for each case (index of the pixel in the healpix map)
        
        that makes 992 * 999 * 25 * 16 = 396403200
        
        then divid this by 1024^3 to get GiB
        
        992*999*25*16/(1024^3)= 0,369179249 GiB

which matches what the code says:

Info moussetDell: Allocating (991008,25) elements = 378.03955078125 MiB in FSRRotation3dMatrix.__init__.

In [ ]:
%matplotlib inline
from pylab import *
import os
import sys
import glob

# Specific science modules
import healpy as hp
import numpy as np
from astropy.io import fits

# Specific qubic modules
from qubicpack.utilities import Qubic_DataDir
from pysimulators import FitsArray
import pysm
import pysm.units as u

import qubic
from qubic import SpectroImLib as si
from qubic import QubicSkySim as qss

from qubic import ReadMC as rmc
from qubic import AnalysisMC as amc

rc('figure', figsize=(13, 12))
rc('font', size=15)

In [ ]:
# Repository for dictionary 
dictfilename = os.environ['QUBIC_DICT']+'spectroimaging_article.dict'

# Read dictionary chosen
d = qubic.qubicdict.qubicDict()
d.read_from_file(dictfilename)

center = qubic.equ2gal(d['RA_center'], d['DEC_center'])
print(center)

d['nf_sub'] = 1
d['Multiband'] = False
d['nf_recon'] = [1]

In [ ]:
maps_dir = '/home/lmousset/QUBIC/MyGitQUBIC/qubic/qubic/scripts/Spectroimagery_paper/maps/'
x0cmb = FitsArray(maps_dir + 'CMB_r=0_nside128_nfsub16.fits')

x0zero = np.zeros_like(x0cmb)
x0ones = np.ones_like(x0cmb)

In [ ]:
save_dir = '/home/lmousset/QUBIC/Qubic_work/SpectroImagerie/paper_simu/Test_kmax/'

# At TOD level
### Vary SynthBeam fraction 

In [ ]:
x0cmb.shape

In [ ]:
d['noiseless'] = True
d['synthbeam_kmax'] = 3
d['npointings'] = 300

# Pointing strategy

p = qubic.get_pointing(d)
p.pitch = p.pitch[0]
seed = 42

x0 = x0ones

allTOD = []
for i in range(10):
    sbfrac = (i + 1) / 10
    print('\n Synthbeam_fraction = ', sbfrac)
    d['synthbeam_fraction'] = sbfrac
    
    np.random.seed(seed) 
    TOD, _ = si.create_TOD(d, p, x0, verbose=True)
    allTOD.append(TOD)

In [ ]:
allTOD = np.array(allTOD)
print(allTOD.shape)

plt.figure()
plt.plot(allTOD[0, 0, :], 'o')

plt.figure()
for i in range(10):
    print(i)
    sbfrac = (i + 1) / 10
    print(sbfrac)
    std_rapport = np.mean(allTOD[i, :, :]) / np.mean(allTOD[9, :, :])
    plt.plot(sbfrac, std_rapport, 'o')
plt.xlabel('SynthBeam fraction')
plt.ylabel('<TOD> / (<TOD> for sb_frac=1) ')
plt.plot([0, 1], [0, 1], 'k')
plt.title('kmax=3, Noiseless=True, x0cmb')
plt.grid()

### Vary kmax

In [ ]:
# Pointing strategy
p = qubic.get_pointing(d)

seed = 42
d['noiseless'] = True

d['synthbeam_fraction'] = 1

x0 = x0cmb

allTOD = []
for i in range(6):
    d['synthbeam_kmax'] = i
    print('\n kmax =', i)  
    
    np.random.seed(seed) 
    TOD, _ = si.create_TOD(d, p, x0, verbose=True)
    allTOD.append(TOD)


In [ ]:
allTOD = np.array(allTOD)
allTOD.shape

plt.figure()
for i in range(0, 6):
    plt.plot(np.mean(allTOD[i, :, :], axis=0), 'o', label='kmax = {}'.format(i))

plt.xlabel('Pointing index')
plt.ylabel('TOD averaged over TES')
plt.xlim(0, 600)
plt.title('sb_frac=1, Noiseless=True, x0cmb')
plt.grid()
plt.legend()


# Mean and STD
all_TODmean = np.mean(allTOD, axis=(1,2))
all_TODstd = np.std(allTOD, axis=(1,2))

plt.figure()
plt.subplot(121)
plt.plot(all_TODmean, 'o')
plt.xlabel('kmax')
plt.ylabel('TOD mean')
plt.grid()

plt.subplot(122)
plt.plot(all_TODstd, 'o')
plt.xlabel('kmax')
plt.ylabel('TOD std')
plt.grid()

plt.suptitle('sb_frac=1, Noiseless=True, x0cmb')



# After map making

In [ ]:
# Pointing strategy
p = qubic.get_pointing(d)

seed = 42
d['noiseless'] = False

d['synthbeam_kmax'] = 1

x0 = x0cmb

npix = 12 * d['nside']**2 
maps_recon = np.zeros((10, npix, 3))
maps_conv = np.zeros((10, npix, 3))
cov = np.zeros((10, npix))

nf_sub_rec = 1
d['tol'] = 1e-5

allTOD = []
for i in range(10):
    sbfrac = (i + 1) / 10
    print('\n Synthbeam_fraction = ', sbfrac)
    d['synthbeam_fraction'] = sbfrac
    
    np.random.seed(seed) 
    TOD, _ = si.create_TOD(d, p, x0, verbose=True)
    allTOD.append(TOD)
    
    
    maps_recon[i], cov[i], nus, nus_edge, maps_conv[i] = si.reconstruct_maps(TOD, d, p, nf_sub_rec, x0)
    
    simu_name = 'skycmb_tol=1e-5_kmax=1_sbfrac={}'.format(sbfrac)
    rmc.save_simu_fits(maps_recon[i], cov[i], nus, nus_edge, maps_conv[i],
                       save_dir=save_dir, simu_name=simu_name)

In [ ]:
# Plot simulations varying kmax
plt.figure()
for i in range(0, 9):
    kmax = i
    simu_name = 'skyzero_tol=1e-5_kmax={}_sbfrac=0.99'.format(i)

    f = fits.open(save_dir + simu_name)
    cov = f['COVERAGE'].data
    seenmap = cov > 0.15 * np.max(cov)
    
    maps_recon, maps_conv, maps_diff = rmc.get_maps(save_dir + simu_name)
    
    bin_centers, ang, std_bin, std_profile = amc.std_profile(np.expand_dims(maps_diff, axis=(0, 1))[:, :, seenmap, :], 
                                                        10, d['nside'], center, seenmap)
    
    plt.plot(bin_centers, std_bin[:, 0, 0], label='kmax = {}'.format(i))#simu_name[-10:])
# plt.xlim(0, 7)
# plt.ylim(0.1, 0.2)
plt.xlabel('Angle [deg]')
plt.ylabel('STD')
plt.title('x0zero - sbfrac=0.99 - STD residual profile')
plt.legend()

In [ ]:
# Plot simulations varying sb_frac
plt.figure()
for i in range(9):
    sbfrac = (i + 1) / 10
    simu_name = 'skycmb_tol=1e-5_kmax=1_sbfrac={}'.format(sbfrac)

    f = fits.open(save_dir + simu_name)
    cov = f['COVERAGE'].data
    seenmap = cov > 0.15 * np.max(cov)
    
    maps_recon, maps_conv, maps_diff = rmc.get_maps(save_dir + simu_name)
    
    bin_centers, ang, std_bin, std_profile = amc.std_profile(np.expand_dims(maps_diff, axis=(0, 1))[:, :, seenmap, :], 
                                                        10, d['nside'], center, seenmap)
    
    plt.plot(bin_centers, std_bin[:, 0, 0], label='sb_frac = {}'.format(sbfrac))

plt.xlabel('Angle [deg]')
plt.ylabel('STD')
plt.title('x0cmb - kmax=1 - STD residual profile')
plt.legend(fontsize=10)

## TOD making and map-making with a different kmax

In [ ]:
kmax = 1
kmaxnew = 2
d['noiseless'] = True
d['synthbeam_fraction'] = 1
d['npointings'] = 1000
d['nside'] = 128

# Pointing strategy
p = qubic.get_pointing(d)

# Input sky
x0 = x0cmb

# Make TOD
d['synthbeam_kmax'] = kmax
seed = 42
np.random.seed(seed) 
TOD, _ = si.create_TOD(d, p, x0, verbose=True)

# Map-making
nf_sub_rec = 1
d['tol'] = 1e-4
# With the same kmax
rec, cov, nus, nus_edge, conv = si.reconstruct_maps(TOD, d, p, nf_sub_rec, x0)
# With a different kmax
d['synthbeam_kmax'] = kmaxnew
recnew, covnew, _, _, convnew = si.reconstruct_maps(TOD, d, p, nf_sub_rec, x0)
    


In [ ]:
# Save the simu
simu_name = 'diffkmax_kmax={}-{}_skycmb_noiselessTrue_ptgs1000_tole-4'.format(kmax, kmax)
# rmc.save_simu_fits(rec, cov, nus, nus_edge, conv, save_dir=save_dir, simu_name=simu_name)

simu_namenew = 'diffkmax_kmax={}-{}_skycmb_noiselessTrue_ptgs1000_tole-4'.format(kmax, kmaxnew)
rmc.save_simu_fits(recnew, covnew, nus, nus_edge, convnew, save_dir=save_dir, simu_name=simu_namenew)


In [ ]:
# Open simu already done
kmax = 2
kmaxnew = 1
simu_name = 'diffkmax_kmax={}-{}_skycmb_noiselessTrue_ptgs1000_tole-4'.format(kmax, kmax)
f = fits.open(save_dir + simu_name)
cov = f['COVERAGE'].data

simu_namenew = 'diffkmax_kmax={}-{}_skycmb_noiselessTrue_ptgs1000_tole-4'.format(kmax, kmaxnew)
fnew = fits.open(save_dir + simu_namenew)
covnew = f['COVERAGE'].data
    
rec, conv, diff = rmc.get_maps(save_dir + simu_name)
recnew, convnew, diffnew = rmc.get_maps(save_dir + simu_namenew)

In [ ]:
# seenmap = cov > 0.1 * np.max(cov)
# seenmapnew = covnew > 0.1 * np.max(covnew)

# diff = rec - conv
# diffnew = recnew - convnew

Stokes = ['I', 'Q', 'U']
for i in range(3):
    if i == 0:
        min = -10
        max = 10
    else:
        min = -0.1
        max = 0.1
    hp.gnomview(conv[:, i] * seenmap, rot=center, reso=15, sub=(3, 6, 6*i+1),
                min=min*20, max=max*20,
                notext=True, title='Conv, kmax {}'.format(kmax))
    plt.text(-0.6, 0, Stokes[i], fontsize=30)
    hp.gnomview(convnew[:, i] * seenmapnew, rot=center, reso=15, sub=(3, 6, 6*i+2),
                min=min*20, max=max*20,
                notext=True, title='Conv, kmax {}'.format(kmaxnew))
    hp.gnomview(rec[:, i] * seenmap, rot=center, reso=15, sub=(3, 6, 6*i+3),
                min=min*20, max=max*20,
                notext=True, title='Rec, kmax {}'.format(kmax))
    hp.gnomview(recnew[:, i] * seenmapnew, rot=center, reso=15, sub=(3, 6, 6*i+4),
                min=min*20, max=max*20,
                notext=True, title='Rec, kmax {}'.format(kmaxnew))
    hp.gnomview(diff[:, i] * seenmap, rot=center, reso=15, sub=(3, 6, 6*i+5),
                min=min/10, max=max/10,
                notext=True, title='Diff, kmax {}'.format(kmax))
    hp.gnomview(diffnew[:, i] * seenmapnew, rot=center, reso=15, sub=(3, 6, 6*i+6),
                min=min/10, max=max/10,
                notext=True, title='Diff, kmax {}'.format(kmaxnew))
    

In [ ]:
# Difference between reconstructed maps and difference between convolved masp
for i in range(3):
    if i == 0:
        min = -20
        max = 20
    else:
        min = -0.1
        max = 0.1
    hp.gnomview(conv[:, i] * seenmap - convnew[:, i] * seenmapnew, rot=center, reso=15, sub=(3, 2, 2*i+1),
                min=min, max=max, notext=True, title='Conv {} - Conv {}'.format(kmax, kmaxnew))
    plt.text(-0.55, 0, Stokes[i], fontsize=30)
    
    hp.gnomview(rec[:, i] * seenmap - recnew[:, i] * seenmapnew, rot=center, reso=15, sub=(3, 2, 2*i+2),
                min=min, max=max, notext=True, title='Rec {} - Rec {}'.format(kmax, kmaxnew))
    

In [ ]:
std = np.zeros(3)
stdnew = np.zeros(3)
for i in range(3):
    std[i] = np.std(diff[seenmap, i])
    stdnew[i] = np.std(diffnew[seenmapnew, i])
print(std, stdnew)